In [ ]:
!pip install beautifulsoup4 deep-translator tqdm pandas

In [ ]:
import time
import os
import re
import logging
from pathlib import Path

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException

# --- Configuration ---

# !IMPORTANT! Set this to the path of your chromedriver.exe
# e.g., "chromedriver.exe" if it's in the same folder
CHROME_DRIVER_PATH = None

# The search results page you provided
START_URL = "https://ted.europa.eu/en/search/result?FT=adalimumab&notice-type=can-standard%2Ccan-social%2Ccan-desg%2Ccan-tran&search-scope=ALL"

# Folder where all the HTML detail pages will be saved
OUTPUT_DIR = "ted_html_pages"

# Safety limit for pagination (961 results / 50 per page ≈ 20 pages)
MAX_PAGES_TO_SCRAPE = 30
HEADLESS = True # Set to False to watch the browser work
PAGE_LOAD_TIMEOUT = 20 # seconds
DOWNLOAD_DELAY = 1.0 # seconds to pause between downloads

# Setup logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s [%(levelname)s] %(message)s")
logger = logging.getLogger(__name__)

# --- Selenium Driver Helper ---

def make_driver(headless=HEADLESS):
    """
    Create Chrome webdriver with Selenium 4 syntax.
    """
    opts = Options()
    if headless:
        opts.add_argument("--headless=new")
    opts.add_argument("--window-size=1600,900")
    opts.add_argument("--disable-gpu")
    opts.add_argument("--log-level=3") # Suppress console noise

    try:
        if CHROME_DRIVER_PATH:
            service = Service(executable_path=CHROME_DRIVER_PATH)
            driver = webdriver.Chrome(service=service, options=opts)
        else:
            logger.warning("CHROME_DRIVER_PATH is not set. Trying automatic driver.")
            driver = webdriver.Chrome(options=opts)
    except Exception as e:
        logger.error(f"Failed to initialize WebDriver: {e}")
        logger.error("Please ensure 'CHROME_DRIVER_PATH' is set correctly.")
        logger.error("And your 'chromedriver.exe' version matches your Chrome browser version.")
        return None

    driver.implicitly_wait(5)
    return driver

# --- File Saving Helper ---

def save_html_content(content: str, filepath: Path):
    """Saves string content to a file."""
    try:
        with open(filepath, "w", encoding="utf-8") as f:
            f.write(content)
        logger.info(f"Saved HTML: {filepath.name}")
    except Exception as e:
        logger.error(f"Failed to save file {filepath.name}: {e}")

# --- Main Scraper Function ---

def scrape_ted_website():
    """
    Main function to scrape the TED website.
    - Handles cookie banner
    - Paginates through results
    - Opens each detail link in a new tab
    - Saves the HTML of the detail page
    """

    # 1. Initialize Driver and Output Dir
    driver = make_driver(headless=HEADLESS)
    if not driver:
        return

    wait = WebDriverWait(driver, PAGE_LOAD_TIMEOUT)
    output_path = Path(OUTPUT_DIR)
    output_path.mkdir(exist_ok=True)

    logger.info(f"WebDriver initialized. Output folder: {OUTPUT_DIR}")

    # 2. Go to Start URL and Handle Cookie Banner
    try:
        driver.get(START_URL)
        # Wait for the cookie banner and accept
        cookie_button = wait.until(
            EC.element_to_be_clickable((By.XPATH, "//button[contains(., 'Accept all')]"))
        )
        cookie_button.click()
        logger.info("Cookie banner accepted.")
    except TimeoutException:
        logger.warning("Cookie banner not found or timed out. Continuing...")
    except Exception as e:
        logger.error(f"Error loading start page or handling cookies: {e}")
        driver.quit()
        return

    # 3. Pagination and Scraping Loop
    page_count = 1
    all_notice_ids = set()

    while page_count <= MAX_PAGES_TO_SCRAPE:
        logger.info(f"--- Processing Results Page {page_count} ---")

        try:
            # Wait for the result links to be present
            wait.until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "a[href*='/notice/-/detail/']"))
            )
            time.sleep(1.0) # Give JS a moment to settle

            # Find all unique detail links on the current page
            links_on_page = driver.find_elements(By.CSS_SELECTOR, "a[href*='/notice/-/detail/']")
            hrefs = list(set([link.get_attribute("href") for link in links_on_page if link.get_attribute("href")]))

            if not hrefs:
                logger.warning("No detail links found on this page. Waiting...")
                time.sleep(5) # Wait longer, maybe page is slow
                links_on_page = driver.find_elements(By.CSS_SELECTOR, "a[href*='/notice/-/detail/']")
                hrefs = list(set([link.get_attribute("href") for link in links_on_page if link.get_attribute("href")]))
                if not hrefs:
                    logger.error("Still no links found. Breaking loop.")
                    break

            logger.info(f"Found {len(hrefs)} unique detail links on page {page_count}.")

            # 4. Loop through links and download HTML
            for href in hrefs:
                try:
                    # Extract notice ID for filename, e.g., "707393-2025"
                    match = re.search(r'/detail/([^/]+)', href)
                    if not match:
                        logger.warning(f"Could not parse notice ID from link: {href}")
                        continue

                    notice_id = match.group(1)

                    # Check if file already exists on disk (from previous runs)
                    filename = f"{notice_id}.html"
                    filepath = output_path / filename
                    if filepath.exists():
                        logger.info(f"File exists, skipping: {filename}")
                        all_notice_ids.add(notice_id) # Add to set to track for this run
                        continue

                    # Check if we've already seen this ID in *this* run (e.g., duplicate link on same page)
                    if notice_id in all_notice_ids:
                        logger.info(f"Already processed {notice_id} in this session, skipping.")
                        continue

                    # Open link in a new tab
                    driver.execute_script("window.open(arguments[0], '_blank');", href)
                    driver.switch_to.window(driver.window_handles[1])

                    # Wait for a key element on the detail page to load
                    wait.until(EC.presence_of_element_located((By.ID, "notice")))

                    # Get page source and save
                    html_content = driver.page_source
                    save_html_content(html_content, filepath)
                    all_notice_ids.add(notice_id) # Add to set after successful save

                    time.sleep(DOWNLOAD_DELAY) # Be polite

                    # Close the new tab and switch back
                    driver.close()
                    driver.switch_to.window(driver.window_handles[0])

                except Exception as e:
                    logger.error(f"Error processing detail page {href}: {e}")
                    # If something went wrong, close the extra tab if it's still open
                    if len(driver.window_handles) > 1:
                        driver.close()
                        driver.switch_to.window(driver.window_handles[0])
                    continue

            # 5. Find and click the "Next" button
            logger.info("Looking for 'Next Page' button...")
            # Updated selector based on user-provided HTML for the <button> element
            next_button_selector = (By.CSS_SELECTOR, "button[aria-label='Go to the next page']")
            next_button = wait.until(
                EC.element_to_be_clickable(next_button_selector)
            )

            # Check if the button is disabled (it will have a 'disabled' attribute)
            if next_button.get_attribute("disabled") is not None:
                logger.info("Next button is disabled. Reached the last page.")
                break

            # --- FIX for ElementClickInterceptedException ---
            # Scroll to the element and use JavaScript click to avoid interception
            try:
                driver.execute_script("arguments[0].scrollIntoView(true);", next_button)
                time.sleep(0.5) # Wait for scroll
                driver.execute_script("arguments[0].click();", next_button)
                logger.info("Clicked 'Next Page' using JavaScript.")
            except Exception as click_err:
                logger.error(f"Failed to click 'Next Page' button even with JS: {click_err}")
                break
            # --- END FIX ---

            page_count += 1
            time.sleep(2.0) # Wait for page transition

        except (TimeoutException, NoSuchElementException):
            logger.info("No 'Next Page' button found or timed out. Assuming end of results.")
            break
        except Exception as e:
            logger.error(f"An unexpected error occurred on page {page_count}: {e}")
            break

    # 6. Clean up
    driver.quit()
    logger.info(f"Scraping complete. Total pages: {page_count-1}. Total unique notices: {len(all_notice_ids)}.")

if __name__ == "__main__":
    scrape_ted_website()



In [6]:
import csv
import os
from bs4 import BeautifulSoup
try:
    from deep_translator import GoogleTranslator
except ImportError:
    print("Error: 'deep_translator' library not found.")
    print("Please install it by running: pip install deep-translator")
    exit()

try:
    from tqdm import tqdm
except ImportError:
    print("Error: 'tqdm' library not found.")
    print("Please install it by running: pip install tqdm")
    print("Progress bar will not be shown.")
    # Create a dummy tqdm function if not installed
    def tqdm(iterable, *args, **kwargs):
        return iterable
    # Create a dummy write function
    tqdm.write = print

def is_numeric_or_empty(text):
    """Checks if text is purely numeric, currency, or empty."""
    if not text:
        return True
    return all(c in '0123456789 ,.' for c in text.strip())

def extract_key_value_from_div_no_translate(div):
    """
    Extracts original key and value from a single div without translating.
    Returns (key_original, value_original) or (None, None)
    """
    key_span = div.find('span', class_='label')
    key_orig = ''
    val_orig = ''

    # PATTERN 1: Standard key-value (span.label)
    if key_span and key_span.parent == div:
        key_orig = key_span.get_text(strip=True)

        # PATTERN 1a: Value in 'data' span
        data_spans = div.find_all('span', class_='data')
        if data_spans:
            val_orig = ' '.join(s.get_text(strip=True) for s in data_spans)
            return key_orig, val_orig

        # PATTERN 1b: Value in another 'label' span
        all_label_spans = div.find_all('span', class_='label')
        if len(all_label_spans) == 2:
            val_orig = all_label_spans[1].get_text(strip=True)
            return key_orig, val_orig

        # PATTERN 1c: Value in 'line' span
        line_span = div.find('span', class_='line')
        if line_span:
            val_orig = line_span.get_text(strip=True)
            return key_orig, val_orig

        return key_orig, "" # Key with no value

    # PATTERN 2: Simple bold span (like "Winner:") or "Tender:"
    # *** BUG FIX ***: Added 'and not data_span' to prevent this from
    # capturing key-value pairs that PATTERN 3 should handle.
    bold_span = div.find('span', class_='bold')
    data_span = div.find('span', class_='data') # Check for data span
    if bold_span and bold_span.parent == div and not key_span and not data_span: # and no label span AND NO DATA SPAN
        # Check if there are other spans (like the ":&nbsp;")
        all_spans = div.find_all('span')
        key_orig = ' '.join(s.get_text(strip=True) for s in all_spans)
        return key_orig.strip(), "" # Return the combined text as key, no value

    # PATTERN 3: Top-level section key-value (e.g., 6.1. Result lot identifier)
    if 'sublevel__content' in div.get('class', []):
        key_span = div.find('span', class_='bold')
        data_span = div.find('span', class_='data')
        if key_span and data_span:
            key_orig = key_span.get_text(strip=True)
            val_orig = data_span.get_text(strip=True)
            return key_orig, val_orig

    return None, None

def extract_all_data_from_section(section_soup):
    """
    Scans one 'section-content' block and extracts all key-value pairs.
    Returns:
        - list_of_key_values: [('key_orig', 'val_orig'), ...]
        - set_of_texts: {'text_to_translate', ...}
    """
    structured_data = [] # To store tuples: [('key_orig', 'val_orig'), ...]
    texts_to_translate = set() # Use a set for unique strings

    main_num_span = section_soup.find('div', class_='sublevel__number', recursive=False)
    main_num = main_num_span.get_text(strip=True) if main_num_span else ''
    main_title = "Main Section" # Default title

    structured_data.append(('Section Number', main_num))
    texts_to_translate.add('Section Number')

    # Process top-level divs
    top_level_divs = section_soup.find_all('div', recursive=False)
    for div in top_level_divs:
        key_orig, val_orig = extract_key_value_from_div_no_translate(div)
        if key_orig is not None:
            if 'sublevel__content' in div.get('class', []):
                main_title = key_orig # Update title
            structured_data.append((key_orig, val_orig))
            if not is_numeric_or_empty(key_orig): texts_to_translate.add(key_orig)
            if not is_numeric_or_empty(val_orig): texts_to_translate.add(val_orig)

    structured_data.append(('Section Title', main_title))
    texts_to_translate.add('Section Title')


    # Process subsections
    subsections = section_soup.find_all('div', class_='subsection-content')
    for sub in subsections:
        sub_num = sub.find('div', class_='sublevel__number').get_text(strip=True)
        sub_title_orig = sub.find('div', class_='sublevel__content').get_text(strip=True)

        structured_data.append((f"{sub_num} Title", sub_title_orig))
        texts_to_translate.add(f"{sub_num} Title")
        if not is_numeric_or_empty(sub_title_orig): texts_to_translate.add(sub_title_orig)

        # --- MODIFIED LOGIC: Iterate direct children for spans OR divs ---
        processed_children = set() # Keep track of divs processed by PATTERN B

        all_children = sub.find_all(recursive=False)
        for child in all_children:

            if child in processed_children or child.name is None:
                continue # Skip if already processed or not a tag

            key_orig, val_orig = None, None

            # Check for sublevel-section
            if 'sublevel-section' in child.get('class', []):
                # This div contains multiple items. Iterate its children.
                sub_children = child.find_all(recursive=False)

                for sub_child in sub_children:
                    if sub_child.name is None: continue

                    key_orig, val_orig = None, None # Reset

                    # It might have a bold span first, which is a sub-sub-title
                    if sub_child.name == 'span' and 'bold' in sub_child.get('class', []):
                        key_orig = sub_child.get_text(strip=True) # Store the title
                        val_orig = "" # This is a title, no value

                    # It will contain divs with key-value pairs
                    elif sub_child.name == 'div':
                        key_orig, val_orig = extract_key_value_from_div_no_translate(sub_child)

                    if key_orig is not None:
                        structured_data.append((key_orig, val_orig))
                        if not is_numeric_or_empty(key_orig): texts_to_translate.add(key_orig)
                        if not is_numeric_or_empty(val_orig): texts_to_translate.add(val_orig)

                processed_children.add(child) # Mark this whole sublevel as processed

            # PATTERN B: Loose spans (like "Non-winning tenderers")
            elif (child.name == 'span' and 'bold' in child.get('class', [])):

                # Find the next sibling that is a 'div'
                next_div = None
                current_sibling = child.next_sibling
                while current_sibling:
                    if current_sibling.name == 'div':
                        next_div = current_sibling
                        break
                    current_sibling = current_sibling.next_sibling

                if next_div:
                    key_orig = child.get_text(strip=True)
                    # Find the value in the next div
                    val_key, val_val = extract_key_value_from_div_no_translate(next_div)
                    if val_key:
                        val_orig = f"{val_key}: {val_val}"
                    else:
                        val_orig = next_div.get_text(strip=True)

                    # Mark this div as processed so PATTERN A doesn't get it
                    processed_children.add(next_div)

            # PATTERN A: Standard div containers (checks for divs)
            elif child.name == 'div':
                key_orig, val_orig = extract_key_value_from_div_no_translate(child)

            # If a key was found, add it
            if key_orig is not None:
                structured_data.append((key_orig, val_orig))
                if not is_numeric_or_empty(key_orig): texts_to_translate.add(key_orig)
                if not is_numeric_or_empty(val_orig): texts_to_translate.add(val_orig)

    return structured_data, texts_to_translate


# --- Main Execution ---

# Initialize the translator
try:
    translator = GoogleTranslator(source='auto', target='en')
except Exception as e:
    print(f"Failed to initialize translator: {e}")
    print("Proceeding without translation.")
    translator = None

# --- Main logic ---

# --- Configuration ---
# Set the path to your HTML folder here
# Use '.' for the current directory
HTML_FOLDER_PATH = '/content/med_webscraper/ted_html_pages'
# --- End Configuration ---

all_headers = set(['file_name'])
all_section_data = [] # List of dictionaries, one per *section*

# Get folder path
folder_path = HTML_FOLDER_PATH # Use the variable

print(f"Scanning folder: {folder_path} ...")
all_html_files = []
for root, dirs, files in os.walk(folder_path):
    for file in files:
        if file.endswith('.html'):
            all_html_files.append(os.path.join(root, file))

file_count = len(all_html_files)
relevant_section_count = 0

if file_count == 0:
    print("No HTML files found in that directory.")
    exit()

print(f"Found {file_count} HTML files. Starting processing...")

for file_path in tqdm(all_html_files, desc="Processing files", unit="file"):
    file_name = os.path.basename(file_path)
    # Use tqdm.write for logging inside the loop to avoid breaking the bar
    tqdm.write(f"\nProcessing file: {file_path}")

    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            html_doc = f.read()
    except Exception as e:
        tqdm.write(f"  Error reading file: {e}")
        continue

    soup = BeautifulSoup(html_doc, 'html.parser')

    main_sections = soup.find_all('div', class_='section-content')
    keyword_found_in_file = False

    for section in main_sections:
        # Check for keyword in the entire block text
        section_text = section.get_text()
        if "adalimumab" in section_text.lower() or section.find('mark', class_='highlight'):

            tqdm.write("  Keyword found in section. Extracting data...")
            keyword_found_in_file = True
            relevant_section_count += 1

            section_kvs, section_texts = extract_all_data_from_section(section)

            # --- Batch Translate for this section ---
            translation_map = {}
            if not translator or not section_texts:
                tqdm.write("    No translator or no text to translate. Using original text.")
            else:
                tqdm.write(f"    Making batch translation request for {len(section_texts)} unique strings...")
                try:
                    text_list = list(section_texts)
                    translated_list = translator.translate_batch(text_list)
                    translation_map = dict(zip(text_list, translated_list))
                    tqdm.write("    Translation complete.")
                except Exception as e:
                    tqdm.write(f"    Warning: Batch translation failed. Using original text. Error: {e}")

            # --- Build the dictionary for this SECTION ---
            section_data_dict = {'file_name': file_name}
            for key_orig, val_orig in section_kvs:
                key_en = translation_map.get(key_orig, key_orig)
                val_en = translation_map.get(val_orig, val_orig)

                # --- FIX: Check if the translated key is None or empty ---
                if not key_en: # This checks for both None and ''
                    continue # Skip this key-value pair entirely

                # --- NEW LOGIC: Append to list instead of overwriting ---
                if key_en not in section_data_dict:
                    # First time seeing this key, create a list
                    section_data_dict[key_en] = [val_en]
                else:
                    # Key already exists, append the new value if it's not already there
                    if val_en not in section_data_dict[key_en]:
                        section_data_dict[key_en].append(val_en)

                all_headers.add(key_en) # Add to global set of headers

            all_section_data.append(section_data_dict)

    if not keyword_found_in_file:
        tqdm.write("  Keyword not found in any section. Skipping file.")
        continue

# This check is now outside the loop
if not all_section_data:
    print("No files containing the keyword were found.")
    exit()

# --- Write final CSV ---
print("\nAll files processed. Writing to 'tender_output.csv'...")

sorted_headers = sorted(list(all_headers - {'file_name'}))
final_headers = ['file_name'] + sorted_headers

try:
    with open('tender_output.csv', 'w', newline='', encoding='utf-8') as f:
        writer = csv.DictWriter(f, fieldnames=final_headers)
        writer.writeheader()

        for section_data in all_section_data:
            # --- NEW: Process lists into strings for CSV ---
            row_to_write = {}
            for header_key in final_headers:
                # Get the value for this header from the section's data
                value = section_data.get(header_key)

                if isinstance(value, list):
                    # Join list items with a " | " separator
                    row_to_write[header_key] = " | ".join(str(v) for v in value)
                elif value:
                    # This will be file_name
                    row_to_write[header_key] = value
                else:
                    # Key doesn't exist for this row, or value is None/empty
                    row_to_write[header_key] = ""

            writer.writerow(row_to_write)

    print(f"Extraction complete. Processed {file_count} files.")
    print(f"Found {relevant_section_count} relevant sections and saved data to 'tender_output.csv'.")

except PermissionError:
    print("\nError: Permission denied. Could not write to 'tender_output.csv'.")
    print("Please make sure the file is not open in another program (like Excel) and you have write permissions.")
except Exception as e:
    print(f"\nAn unexpected error occurred while writing the CSV: {e}")

Scanning folder: /content/med_webscraper/ted_html_pages ...
Found 961 HTML files. Starting processing...


Processing files:   0%|          | 0/961 [00:00<?, ?file/s]


Processing file: /content/med_webscraper/ted_html_pages/146433-2022.html


Processing files:   0%|          | 3/961 [00:00<02:14,  7.14file/s]

  Keyword not found in any section. Skipping file.

Processing file: /content/med_webscraper/ted_html_pages/551557-2021.html
  Keyword not found in any section. Skipping file.

Processing file: /content/med_webscraper/ted_html_pages/473752-2018.html
  Keyword not found in any section. Skipping file.

Processing file: /content/med_webscraper/ted_html_pages/66455-2021.html


...

  Keyword found in section. Extracting data...
    Making batch translation request for 66 unique strings...


Processing files: 100%|█████████▉| 960/961 [38:22<00:06,  6.14s/file]

    Translation complete.

Processing file: /content/med_webscraper/ted_html_pages/235133-2018.html


Processing files: 100%|██████████| 961/961 [38:22<00:00,  2.40s/file]

  Keyword not found in any section. Skipping file.

All files processed. Writing to 'tender_output.csv'...
Extraction complete. Processed 961 files.
Found 267 relevant sections and saved data to 'tender_output.csv'.


**Cleaning**

In [14]:
import pandas as pd
import re

# Load the CSV file
try:
    df = pd.read_csv('tender_output.csv')
except FileNotFoundError:
    print("Error: 'tender_output.csv' not found. Please run the previous steps to generate the file.")
    exit()

# Define the columns to keep
columns_to_keep = [
    'file_name', # Add 'file_name' to columns to keep
    'Country',
    'Date of the conclusion of the contract',
    'Description',
    'Duration',
    'Estimated value excluding VAT',
    'Lot',
    'Non-winning tenderers',
    'Official name', # Keep 'Official name' temporarily for renaming
    'Review organisation',
    'Start date',
    'Title',
    'Value of the tender',
    'Main classification', # Keep 'Main classification' temporarily for CPV processing
    'Winner selection status' # Keep 'Winner selection status' temporarily for filtering
]

# Filter the DataFrame to keep only the specified columns
# Ensure all columns_to_keep exist in the DataFrame
existing_columns = [col for col in columns_to_keep if col in df.columns]
df_filtered = df[existing_columns].copy()

# Clean the 'Main classification' column to only keep numerical code
if 'Main classification' in df_filtered.columns:
    df_filtered['Main classification'] = df_filtered['Main classification'].apply(lambda x: re.search(r'\d+', str(x)).group(0) if re.search(r'\d+', str(x)) else '')
    # Rename the cleaned column to 'CPV'
    df_filtered.rename(columns={'Main classification': 'CPV'}, inplace=True)
    # Add 'CPV' to the list of columns to display if it wasn't already there
    if 'CPV' not in columns_to_keep:
        columns_to_keep.append('CPV') # Add 'CPV' to the display list

# Rename 'Official name' to 'Winner'
if 'Official name' in df_filtered.columns:
    df_filtered.rename(columns={'Official name': 'Winner'}, inplace=True)
    # Update columns_to_keep to reflect the renaming for the final output
    if 'Official name' in columns_to_keep:
        columns_to_keep.remove('Official name')
    if 'Winner' not in columns_to_keep:
        columns_to_keep.append('Winner')


# Fill empty 'Value of the tender' with 'Estimated value excluding VAT' and mark with '*'
if 'Value of the tender' in df_filtered.columns and 'Estimated value excluding VAT' in df_filtered.columns:
    mask = df_filtered['Value of the tender'].isna() | (df_filtered['Value of the tender'] == '')
    df_filtered.loc[mask, 'Value of the tender'] = '*' + df_filtered.loc[mask, 'Estimated value excluding VAT'].astype(str)

# Function to convert currency string to float and keep currency code
def convert_currency_to_float(currency_str):
    if pd.isna(currency_str) or currency_str == '':
        return None
    # Remove the '*' if present
    currency_str = str(currency_str).lstrip('*').strip()
    if not currency_str:
        return None

    # Use regex to find the currency code and the numerical part
    # This regex handles currency codes before or after the number
    match = re.search(r'([A-Z]{3})?\s*([\d\s,.]*)?\s*([A-Z]{3})?', currency_str)
    if match:
        currency_code_before = match.group(1)
        number_part = match.group(2)
        currency_code_after = match.group(3)

        currency_code = currency_code_before if currency_code_before else (currency_code_after if currency_code_after else '')

        if not number_part:
          return currency_str # Return original if no number part found


        # Clean the number part: remove spaces and identify decimal separator
        cleaned_number_str = number_part.replace(' ', '')

        # Determine decimal separator: if there's a comma AND a dot, it's ambiguous.
        # Assuming last separator is decimal if both exist, otherwise use the single one.
        if ',' in cleaned_number_str and '.' in cleaned_number_str:
            if cleaned_number_str.rfind(',') > cleaned_number_str.rfind('.'):
                # Comma is the decimal separator, dot is thousand separator
                cleaned_number_str = cleaned_number_str.replace('.', '').replace(',', '.')
            else:
                # Dot is the decimal separator, comma is thousand separator
                cleaned_number_str = cleaned_number_str.replace(',', '')
        elif ',' in cleaned_number_str:
            # Only comma present, assume it's the decimal separator
            cleaned_number_str = cleaned_number_str.replace(',', '.')
        # If only dot is present, assume it's the decimal separator (no change needed)
        # If no separator is present, it's an integer (no change needed)


        try:
            float_value = float(cleaned_number_str)
            return f"{float_value} {currency_code}".strip()
        except ValueError:
            # If conversion still fails, return the original cleaned string for inspection
            return f"ERROR: {cleaned_number_str} {currency_code}".strip() # Return original if conversion fails
    return currency_str # Return original if no match

# Apply the conversion function to create the new 'final_value' column
if 'Value of the tender' in df_filtered.columns:
    df_filtered['final_value'] = df_filtered['Value of the tender'].apply(convert_currency_to_float)
    # Add 'final_value' to the list of columns to display
    if 'final_value' not in columns_to_keep:
        columns_to_keep.append('final_value')


# Filter rows based on "Winner selection status" or "Estimated value excluding VAT"
# Remove the temporary columns after filtering if they were added just for filtering
final_columns = [col for col in columns_to_keep if col in df_filtered.columns and col not in ['Winner selection status', 'Official name', 'Main classification']]


df_final = df_filtered[
    (df_filtered['Winner selection status'].notna() & (df_filtered['Winner selection status'] != '')) |
    (df_filtered['Estimated value excluding VAT'].notna() & (df_filtered['Estimated value excluding VAT'] != ''))
][final_columns]


# Display the first few rows of the processed DataFrame
display(df_final.head())

# Optionally, save the processed DataFrame to a new CSV file
df_final.to_csv('processed_tender_output.csv', index=False)
print("\nProcessed data saved to 'processed_tender_output.csv'")

,file_name,Country,Date of the conclusion of the contract,Description,Duration,Estimated value excluding VAT,Lot,Non-winning tenderers,Review organisation,Start date,Title,Value of the tender,CPV,Winner,final_value
2,111063-2025.html,Spain,NaN,Award of the Contract Based on the Health Corp...,NaN,"973 236,00 EUR",NaN,NaN,NaN,NaN,Award of the Contract Based on the Framework A...,"*973 236,00 EUR",33690000,NaN,973236.0 EUR
4,15498-2025.html,Germany,NaN,Reference is made to the respective order anno...,NaN,"1,00 EUR",LOT-0001,NaN,Federal Public Procurement Chambers at the Fed...,NaN,See the “General Information” field,"*1,00 EUR",33600000,NaN,1.0 EUR
6,357325-2024.html,Spain,NaN,Adalimumab20 MG - Vial/Prefilled Syringe/Pen,NaN,"121 888,80 EUR",LOT-0001,NaN,General Directorate of Economic Resources of t...,NaN,Adalimumab20 MG,"*121 888,80 EUR",33690000,NaN,121888.8 EUR
7,357325-2024.html,Spain,NaN,Adalimumab40 mg - Vial/Pre-filled syringe/Pen,NaN,"37 830 672,00 EUR",LOT-0002,NaN,General Directorate of Economic Resources of t...,NaN,Adalimumab40 mg,"*37 830 672,00 EUR",33690000,NaN,37830672.0 EUR
8,357325-2024.html,Spain,NaN,Adalimumab80 mg - Vial/Prefilled Syringe/Pen,NaN,"1 390 435,20 EUR",LOT-0003,NaN,General Directorate of Economic Resources of t...,NaN,Adalimumab 80 mg,"*1 390 435,20 EUR",33690000,NaN,1390435.2 EUR



Processed data saved to 'processed_tender_output.csv'
